Baseline model on Both the D1 and D2 Dataset

For our purpose we have used the Data and models stored in Drive. But we have also provided the models in Github which can be imported and used here and provided the drive link too.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import multiprocessing as mp
import cv2
from tqdm import tqdm_notebook
from tqdm import tnrange
import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms
import os
from PIL import Image
import torch
import torch.nn 
import torchvision
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**LOADING THE DATA**

In [ ]:
lxtrain=np.load("/content/drive/MyDrive/urbansound8k/300data/final/xtrainfinal.npy")
lytrain=np.load("/content/drive/MyDrive/urbansound8k/300data/final/ytrainfinal.npy")
lxtest=np.load("/content/drive/MyDrive/urbansound8k/300data/final/xtestfinal.npy")
lytest=np.load("/content/drive/MyDrive/urbansound8k/300data/final/ytestfinal.npy")

**PROTOTYPICAL NETWORKS**

In [ ]:
def l2norm(X):
    norm = torch.pow(X, 2).sum(dim=-1, keepdim=True).sqrt()
    X = torch.div(X, norm)
    return X

In [ ]:
def euclidean_dist(x, y):
  a = x.size(0)
  b = y.size(0)
  c = x.size(1)
  x = x.unsqueeze(1).expand(a, b, c)
  y = y.unsqueeze(0).expand(a, b, c)

  return torch.pow(x - y, 2).sum(2)

In [ ]:
class Flattening(torch.nn.Module):
  def __init__(self):
    super(Flattening, self).__init__()

  def forward(self, x):
    return x.view(x.size(0), -1)

In [ ]:
def Protonet_conv2d(xshape, hiddenshape, yshape):
  embed = torch.nn.Sequential(
    torch.nn.Conv2d(xshape[0], hiddenshape, 3, padding=1),
    torch.nn.BatchNorm2d(hiddenshape),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(3),
    torch.nn.Conv2d(hiddenshape, hiddenshape, 3, padding=1),
    torch.nn.BatchNorm2d(hiddenshape),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(3),
    torch.nn.Conv2d(hiddenshape, hiddenshape, 3, padding=1),
    torch.nn.BatchNorm2d(hiddenshape),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(3),
    torch.nn.Conv2d(hiddenshape, yshape, 3, padding=1),
    torch.nn.BatchNorm2d(yshape),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(3),
    Flattening()
    )
    
  return load_samples(embed)

In [ ]:
class load_samples(torch.nn.Module):
  def __init__(self, embed):
    
    super(load_samples, self).__init__()
    self.embed = embed.cuda()

  def loss_and_pred(self, spectograms):
    specs = spectograms['specs'].cuda()
    n_way = spectograms['n_way']
    n_support = spectograms['support']
    n_query = spectograms['query']
    support_data = specs[:, :support]
    labels = torch.arange(0, n_way).view(n_way, 1, 1).expand(n_way, query, 1).long()
    query_data = specs[:, support:]
    labels = Variable(labels, requires_grad=False)
    #print(labels)
    
    labels = labels.cuda()
    x = torch.cat([support_data.contiguous().view(n_way * support, *support_data.size()[2:]),query_data.contiguous().view(n_way * n_query, *query_data.size()[2:])], 0)
    y = self.embed.forward(x)
    yshape = y.size(-1) 
    query_y = y[n_way*support:]
    prototype = y[:n_way*support].view(n_way, support, yshape).mean(1)
    #unlabel_prototype = 
    distance = euclidean_dist(query_y, prototype)
    # print("k")
    # print(n_way)
    probab = F.log_softmax(-distance,dim=1).view(n_way, query, -1)
   
    lossprobab = -probab.gather(2, labels).squeeze().view(-1).mean()
    m, yk = probab.max(2)
    accuracy = torch.eq(yk, labels.squeeze()).float().mean()
   
    return lossprobab, {'loss': lossprobab.item(),'acc': accuracy.item(),'yk': yk}

**SECTION FOR TRAINING TESTING ON DATASET D1 URBANSOUND8K AND FSD DATASET**

In [ ]:
def train(model, optimizer, train_x, train_y, n_way, support, query, max_epoch, epoch_size):
  #divide the learning rate by 2 at each epoch, as suggested in paper
  scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5, last_epoch=-1)
  epoch = 0 #epochs done so far
  while epoch < max_epoch:
    loss_total = 0.0
    accuracy = 0.0

    for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch+1)):
      batch = []
      K = np.random.choice(np.unique(train_y), n_way, replace = False)
      for cls in K:
        datax_cls = train_x[train_y == cls]
        perm = np.random.permutation(datax_cls)
        batch_cls = perm[:(support+query)]
        batch.append(batch_cls)
      batch = np.array(batch)
      batch = torch.from_numpy(batch).float()
      batch = batch.permute(0,1,4,2,3)
      subset = {'specs': batch,'n_way': n_way,'support': support,'query': query}
      optimizer.zero_grad()
      loss, output = model.loss_and_pred(subset)
      loss_total += output['loss']
      accuracy += output['acc']
      loss.backward()
      optimizer.step()
    epoch_loss = loss_total / epoch_size
    epoch_acc = accuracy / epoch_size
    print('Epoch {:d} -- Loss: {:.4f} Acc: {:.4f}'.format(epoch+1,epoch_loss, epoch_acc))
    epoch += 1
    scheduler.step()
    

In [ ]:
model = Protonet_conv2d(xshape=(4, 130, 130),hiddenshape=64,yshape=64)
optimizer = optim.Adam(model.parameters(), lr = 0.003)

In [ ]:
n_way = 5 # number of classes taken at a time
support = 5 # number of examples in support set
query = 5 # number of examples in query set
max_epoch = 10
epoch_size = 2000

In [ ]:
#torch.save(model,"/content/drive/MyDrive/urbansound8k/300data/final/300data_baseline_pmen.pt")

In [ ]:
train(model, optimizer, lxtrain, lytrain, n_way, support, query, max_epoch, epoch_size)
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
#torch.save(model,"/content/drive/MyDrive/urbansound8k/300data/final/300data_baseline_pmen.pt")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  if __name__ == '__main__':



Epoch 1 -- Loss: 0.5662 Acc: 0.7828



Epoch 2 -- Loss: 0.3382 Acc: 0.8734



Epoch 3 -- Loss: 0.2226 Acc: 0.9182



Epoch 4 -- Loss: 0.1457 Acc: 0.9473


...


Epoch 7 -- Loss: 0.0686 Acc: 0.9752



Epoch 8 -- Loss: 0.0665 Acc: 0.9759



Epoch 9 -- Loss: 0.0608 Acc: 0.9786



Epoch 10 -- Loss: 0.0580 Acc: 0.9793


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1794 KB |     921 MB |   73255 GB |   73255 GB |\n|       from large pool |       0 KB |     919 MB |   73108 GB |   73108 GB |\n|       from small pool |    1794 KB |       3 MB |     146 GB |     146 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |    1794 KB |     921 MB |   73255 GB |   73255 GB |\n|       from large pool |       0 KB |     919 MB |

In [ ]:
model = torch.load('/content/drive/MyDrive/FSD2018/folder/baselineCOMBINE.pt')

**TESTING AND EVALUATION**

In [ ]:
##TESTTTTTTINGGG
def test(model, test_x, test_y, n_way, n_support, n_query, test_episode):
  accuracy = 0.0
  loss_total = 0.0
  for episode in tnrange(test_episode):
    batch = []
    K = np.random.choice(np.unique(test_y), n_way, replace = False)
    for cls in K:
      datax_cls = test_x[test_y == cls]
      perm = np.random.permutation(datax_cls)
      batch_cls = perm[:(support+query)]
      batch.append(batch_cls)
    batch = np.array(batch)
    # print(test_y)
    batch = torch.from_numpy(batch).float()
    batch = batch.permute(0,1,4,2,3)
    subset = {'specs': batch,'n_way': n_way,'support': n_support,'query': n_query}
    # print(subset.shape)
    loss, output = model.loss_and_pred(subset)
    # print(output)
    loss_total += output['loss']
    accuracy += output['acc']
  avg_loss = loss_total / test_episode
  avg_acc = accuracy / test_episode
  print('Test results -- Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))

In [ ]:
n_way =5
support = 5
query = 5
test_x = lxtest
test_y = lytest
test_episode = 5000
test(model, test_x, test_y, n_way, support, query, test_episode)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """



Test results -- Loss: 0.9819 Acc: 0.6697


In [ ]:
baseline result = 66.97 percent acc


SECTION FOR TRAINING TESTING ON DATASET D2 FSD DATASET ONLY

---



In [ ]:
lxtrain=np.load("/content/drive/MyDrive/urbansound8k/300data/xtrainnewFSD.npy")
lytrain=np.load("/content/drive/MyDrive/urbansound8k/300data/ytrainnewFSD.npy")
lxtest=np.load("/content/drive/MyDrive/urbansound8k/300data/xtestnewFSD.npy")
lytest=np.load("/content/drive/MyDrive/urbansound8k/300data/ytestnewFSD.npy")

In [ ]:
model = Protonet_conv2d(xshape=(4, 130, 130),hiddenshape=64,yshape=64)
optimizer = optim.Adam(model.parameters(), lr = 0.003)
n_way = 5 # number of classes taken at a time
support = 5 # number of examples in support set
query = 5 # number of examples in query set
max_epoch = 10
epoch_size = 2000
train(model, optimizer, lxtrain, lytrain, n_way, support, query, max_epoch, epoch_size)
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  if __name__ == '__main__':



Epoch 1 -- Loss: 0.7289 Acc: 0.7307



Epoch 2 -- Loss: 0.3381 Acc: 0.8796



Epoch 3 -- Loss: 0.1632 Acc: 0.9432



Epoch 4 -- Loss: 0.0815 Acc: 0.9723


...


Epoch 7 -- Loss: 0.0281 Acc: 0.9901



Epoch 8 -- Loss: 0.0242 Acc: 0.9922



Epoch 9 -- Loss: 0.0220 Acc: 0.9930



Epoch 10 -- Loss: 0.0213 Acc: 0.9929


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1794 KB |     908 MB |   30326 GB |   30326 GB |\n|       from large pool |       0 KB |     906 MB |   30223 GB |   30223 GB |\n|       from small pool |    1794 KB |       3 MB |     102 GB |     102 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |    1794 KB |     908 MB |   30326 GB |   30326 GB |\n|       from large pool |       0 KB |     906 MB |

In [ ]:
torch.save(model,"/content/drive/MyDrive/urbansound8k/300data/final/baseline_FSD.pt")

In [ ]:
model = torch.load("/content/drive/MyDrive/urbansound8k/300data/final/baseline_FSD.pt")

In [ ]:
##TESTTTTTTINGGG
def test(model, test_x, test_y, n_way, n_support, n_query, test_episode):
  accuracy = 0.0
  loss_total = 0.0
  for episode in tnrange(test_episode):
    batch = []
    K = np.random.choice(np.unique(test_y), n_way, replace = False)
    for cls in K:
      datax_cls = test_x[test_y == cls]
      perm = np.random.permutation(datax_cls)
      batch_cls = perm[:(support+query)]
      batch.append(batch_cls)
    batch = np.array(batch)
    # print(test_y)
    batch = torch.from_numpy(batch).float()
    batch = batch.permute(0,1,4,2,3)
    subset = {'specs': batch,'n_way': n_way,'support': n_support,'query': n_query}
    # print(subset.shape)
    loss, output = model.loss_and_pred(subset)
    # print(output)
    loss_total += output['loss']
    accuracy += output['acc']
  avg_loss = loss_total / test_episode
  avg_acc = accuracy / test_episode
  print('Test results -- Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))

In [ ]:
n_way =5
support = 5
query = 5
test_x = lxtest
test_y = lytest
test_episode = 5000
test(model, test_x, test_y, n_way, support, query, test_episode)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  """



Test results -- Loss: 2.0187 Acc: 0.4387
